In [1]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  18.3M      0  0:00:42  0:00:42 --:--:-- 17.3M


In [2]:
!unzip -q kagglecatsanddogs_5340.zip
!ls


CDLA-Permissive-2.0.pdf              kagglecatsanddogs_5340.zip
CSE_572_-Project-3-Directions-1.docx model.ipynb
PatientData                          readme[1].txt
PatientData-2.zip                    ~$E_572_-Project-3-Directions-1.docx
PetImages


In [2]:
filename = "/Users/tara/Documents/Data mining/Assignment3/PatientData/Training/Noise/1IC_1_thresh.png"
import cv2
cv2.imread(filename).shape


ModuleNotFoundError: No module named 'cv2'

In [20]:
import os
from os import listdir
import shutil

import csv
rsn_dir = "/Users/tara/Documents/Data mining/Assignment3/PatientData/Training/RSN"
noise_dir = "/Users/tara/Documents/Data mining/Assignment3/PatientData/Training/Noise"
if not os.path.exists(rsn_dir):
        os.makedirs(rsn_dir)
        os.makedirs(noise_dir)
for i in range(1,6):
    with open(f'/Users/tara/Documents/Data mining/Assignment3/PatientData/Patient_{i}_Labels.csv', 'r') as file:
        reader = csv.DictReader(file)
        dict = {}
        for row in reader:
            dict[row['IC']] = int(row['Label'])
        # print(dict)
    
    # get the path/directory
    
    folder_dir = f"/Users/tara/Documents/Data mining/Assignment3/PatientData/Patient_{i}"
    images_arr = []
    for image in os.listdir(folder_dir):
        # check if the image ends with png
        if (image.endswith("thresh.png")):
            IC_number = image.split("_")[1]
            label = dict[IC_number]
            if label == 0:
                shutil.copyfile(os.path.join(folder_dir,image),os.path.join(noise_dir,str(i)+image))
            else:
                shutil.copyfile(os.path.join(folder_dir,image),os.path.join(rsn_dir,str(i)+image))


In [1]:

image_size = (851, 1026)
batch_size = 32
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
train_ds, val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/tara/Documents/Data mining/Assignment3/PatientData/Training",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 502 files belonging to 2 classes.
Using 402 files for training.
Using 100 files for validation.


In [2]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)



In [3]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # for size in [256, 512, 728]:
    #     x = layers.Activation("relu")(x)
    #     x = layers.SeparableConv2D(size, 3, padding="same")(x)
    #     x = layers.BatchNormalization()(x)

    #     x = layers.Activation("relu")(x)
    #     x = layers.SeparableConv2D(size, 3, padding="same")(x)
    #     x = layers.BatchNormalization()(x)

    #     x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    #     # Project residual
    #     residual = layers.Conv2D(size, 1, strides=2, padding="same")(
    #         previous_block_activation
    #     )
    #     x = layers.add([x, residual])  # Add back residual
    #     previous_block_activation = x  # Set aside next residual

    # x = layers.SeparableConv2D(256, 3, padding="same")(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)


In [4]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/25


2022-11-23 23:22:10.321722: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


13/13 [==============================] - 318s 24s/step - loss: 0.7070 - accuracy: 0.5274 - val_loss: 0.6937 - val_accuracy: 0.4700
Epoch 2/25
13/13 [==============================] - 307s 23s/step - loss: 0.7090 - accuracy: 0.5149 - val_loss: 0.6930 - val_accuracy: 0.5900
Epoch 3/25
13/13 [==============================] - 347s 26s/step - loss: 0.7095 - accuracy: 0.5448 - val_loss: 0.6935 - val_accuracy: 0.4700
Epoch 4/25
13/13 [==============================] - 313s 24s/step - loss: 0.7008 - accuracy: 0.4851 - val_loss: 0.6932 - val_accuracy: 0.4700
Epoch 5/25
13/13 [==============================] - 321s 25s/step - loss: 0.6958 - accuracy: 0.5274 - val_loss: 0.6925 - val_accuracy: 0.5300
Epoch 6/25
13/13 [==============================] - 330s 25s/step - loss: 0.7129 - accuracy: 0.5124 - val_loss: 0.6929 - val_accuracy: 0.5400
Epoch 7/25
 1/13 [=>............................] - ETA: 6:56 - loss: 0.7107 - accuracy: 0.5625

KeyboardInterrupt: 

In [16]:
import csv
with open('/Users/tara/Documents/Data mining/Assignment3/PatientData/Patient_1_Labels.csv', 'r') as file:
    reader = csv.DictReader(file)
    dict = {}
    for row in reader:
        dict[row['IC']] = int(row['Label'])
    print(dict)


{'1': 0, '2': 0, '3': 0, '4': 0, '5': 1, '6': 0, '7': 0, '8': 0, '9': 1, '10': 0, '11': 0, '12': 1, '13': 1, '14': 0, '15': 0, '16': 0, '17': 1, '18': 0, '19': 0, '20': 1, '21': 0, '22': 0, '23': 1, '24': 0, '25': 1, '26': 0, '27': 0, '28': 1, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 1, '35': 0, '36': 1, '37': 1, '38': 0, '39': 1, '40': 1, '41': 1, '42': 0, '43': 1, '44': 1, '45': 0, '46': 0, '47': 0, '48': 1, '49': 0, '50': 0, '51': 0, '52': 1, '53': 0, '54': 0, '55': 0, '56': 1, '57': 3, '58': 1, '59': 0, '60': 0, '61': 3, '62': 0, '63': 1, '64': 0, '65': 1, '66': 0, '67': 0, '68': 0, '69': 0, '70': 0, '71': 0, '72': 3, '73': 0, '74': 0, '75': 0, '76': 0, '77': 0, '78': 1, '79': 0, '80': 0, '81': 3, '82': 1, '83': 1, '84': 0, '85': 0, '86': 0, '87': 0, '88': 1, '89': 3, '90': 0, '91': 1, '92': 3, '93': 0, '94': 0, '95': 0, '96': 3, '97': 0, '98': 1, '99': 0, '100': 1, '101': 0, '102': 0, '103': 0, '104': 1, '105': 1, '106': 1, '107': 0, '108': 0, '109': 0, '110': 0, '111': 